In [2]:
import sys, os
sys.path.append(os.path.dirname(os.getcwd())) # Include ../SSD in path
import numpy as np
import torch
import matplotlib.pyplot as plt
import pandas as pd
import dataset_exploration.analyze_stuff as analyze
import statistics
from vizer.draw import draw_boxes
from tops.config import instantiate, LazyConfig
from ssd import utils
#import configs.retinaNet_focal_loss as config
np.random.seed(0)

ModuleNotFoundError: No module named 'vizer'

In [3]:
BATCH_SIZE = 1
SET_TYPE = 'train'
CFG_PATH = 'configs/retinaNet_focal_loss.py'

cfg = analyze.get_config(CFG_PATH)
#print(cfg)
metadata = analyze.load_annotation_file(cfg, SET_TYPE)
#number of times a value appears = (df['image_id']==1).sum()
# occourence_classes = df['category_id'].value_counts()
annotations = metadata['annotations']
df = pd.DataFrame(data=annotations)
#df=df.iloc[9000:,:]   #exploring only parts of the data

class_dict = {"car":[], "truck":[], "bus":[], "MC":[], "bicycle":[], "scooter":[], "person":[], "rider":[]}
class_names = ("car", "truck", "bus", "MC", "bicycle", "scooter", "person", "rider")
hights_median = []
widths_median = []
hights_mean = []
widths_mean = []
hights_std = []
widths_std = []
mean_ratio = []
median_ratio = []
areas = []
print('_____________________________BBOX Statistics_______________________')
for i,class_name in enumerate(class_names):
     
     
     df_class = df.loc[df['category_id']==i+1]
     hight = []
     width = []
     x = []
     y = []
     for j in range(len(df_class)):
          list = df_class['bbox'].values[j]
          x.append(list[0])
          y.append(list[1])          
          width.append(list[2])
          hight.append(list[3])
     dict = {'x':x,'y':y,'hight':hight,'width':width}
     df_bbox = pd.DataFrame(dict)      
     bbox_description = df_bbox.describe()
     print('________________________',class_name,'________________________')
     print(bbox_description) # uncomment to see the entire description for each class
     hights_median.append(bbox_description['hight']['50%'])
     widths_median.append(bbox_description['width']['50%'])
     hights_mean.append(bbox_description['hight']['mean'])
     widths_mean.append(bbox_description['width']['mean'])
     hights_std.append(bbox_description['hight']['std'])
     widths_std.append(bbox_description['width']['std'])
     
areas_mean = []
areas_median = []
print('\n','\n','\n','\n','_________________AREA statistics________________________')
for i,class_name in enumerate(class_names):     
     print('________________________',class_name,'________________________')
     df_class = df.loc[df['category_id']==i+1]        
     area_description = df_class['area'].describe()
     areas_mean.append(area_description['mean'])
     areas_median.append(area_description['50%'])
     print(area_description)
     #areas.append(area_description)
for i,class_name in enumerate(class_names):  
     mean_ratio.append(hights_mean[i]/widths_mean[i])    
     median_ratio.append(hights_median[i]/widths_median[i]) 




# Plots
plt.bar(class_names,mean_ratio)
plt.xlabel('class')
plt.ylabel('mean ratio')
plt.title('mean ratio for every class h/w')

path = "analysis_plot/img_{}.eps".format('mean_ratio')
plt.savefig(path, format='eps')
plt.show()

plt.bar(class_names,median_ratio)
plt.xlabel('class')
plt.ylabel('median ratio')
plt.title('median ratio for every class h/w')

path = "analysis_plot/img_{}.eps".format('median_ratio')
plt.savefig(path, format='eps')
plt.show()

plt.bar(class_names,areas_mean)
plt.xlabel('class')
plt.ylabel('mean area')
plt.title('mean area for every class')
path = "analysis_plot/img_{}.eps".format('areas_mean')
plt.savefig(path, format='eps')
plt.show()

plt.bar(class_names,areas_median)
plt.xlabel('class')
plt.ylabel('median area')
plt.title('median area for every class')
path = "analysis_plot/img_{}.eps".format('areas_median')
plt.savefig(path, format='eps')
plt.show()

plt.scatter(widths_median,hights_median)
plt.xlabel('width')
plt.ylabel('height')
plt.title('median hight and width for each class')
for i,txt in enumerate(class_names):
     plt.text(widths_median[i]+0.3,hights_median[i],txt)
path = "analysis_plot/img_{}.eps".format('hights_median')
plt.savefig(path, format='eps')
plt.show()



plt.scatter(widths_mean,hights_mean)
plt.xlabel('width')
plt.ylabel('height')
plt.title('mean hight and width for each class')
for i,txt in enumerate(class_names):
     if txt != 'scooter':
          plt.text(widths_mean[i]+0.5,hights_mean[i]-1,txt)
     else:
          plt.text(widths_mean[i]+0.5,hights_mean[i],txt)
path = "analysis_plot/img_{}.eps".format('hights_mean')
plt.savefig(path, format='eps')
plt.show()

plt.bar(class_names,hights_std)
plt.xlabel('class')
plt.ylabel('std')
plt.title('standard deviation hight')
path = "analysis_plot/img_{}.eps".format('hights_std')
plt.savefig(path, format='eps')
plt.show()

plt.bar(class_names,widths_std)
plt.xlabel('class')
plt.ylabel('std')
plt.title('standard deviation width')
path = "analysis_plot/img_{}.eps".format('widths_std')
plt.savefig(path, format='eps')
plt.show()


occourence_classes = df['category_id'].value_counts()
print(occourence_classes)
plt.bar(occourence_classes.index,occourence_classes.values)
plt.xlabel('class')
plt.ylabel('appearances')
plt.title('number of appearances')
path = "analysis_plot/img_{}.eps".format('occourence_classes')
plt.savefig(path, format='eps')
plt.show()

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\Jonas/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:02<00:00, 43.5MB/s]


['T_destination', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_buffers', '_call_impl', '_forward_hooks', '_forward_pre_hooks', '_get_backward_hooks', '_get_name', '_is_full_backward_hook', '_load_from_state_dict', '_load_state_dict_pre_hooks', '_maybe_warn_non_full_backward_hook', '_modules', '_named_members', '_non_persistent_buffers_set', '_parameters', '_register_load_state_dict_pre_hook', '_register_state_dict_hook', '_replicate_for_data_parallel', '_save_to_state_dict', '_slow_forward', '_state_dict_hooks', '_version', 'add_module', 'apply', 'bfloat16', 'buffers', 'children', 'cpu', 'cuda', 'doub

TypeError: __init__() got an unexpected keyword argument 'flags'